In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from kernel_svm import *
from kernel_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import numpy as np


def sparse_absolute(x):
    res = x.copy()
    temp = np.abs(res.data)
    res.data = temp
    return res


def sparse_scalar_multiplication(x, a):
    res = x.copy()
    res.data = a * res.data
    return res


def sparse_min(x, y):
    res = sparse_absolute(x + y) - sparse_absolute(x - y)
    res = sparse_scalar_multiplication(res, 0.5)
    return res


def sparse_norm_2(x, square_norm=False):
    res = x.power(2).sum()
    if square_norm:
        return res
    else:
        return np.sqrt(res)

In [17]:
import numpy as np

def linear_kernel(x, y):
    res = x.dot(y.transpose()).data[0]
    return res


def polynomial_kernel(x, y, d):
    res = (x.dot(y.transpose()).data[0] + 1) ** d
    return res


def non_gaussian_rbf_kernel(x, y, a, b, gamma):
    res = sparse_absolute(x.power(a) - y.power(a))
    res = res.power(b)
    res = res.sum()
    res = np.exp(-gamma * res)
    return res


def generalized_histogram_kernel(x, y, a, b):
    x_a = x.power(a)
    y_b = y.power(b)
    res = sparse_min(x_a, y_b)
    res = res.sum()
    return res


def gaussian_kernel(x, y, gamma):
    res = np.exp(-gamma * sparse_norm_2(x - y, True))
    return res


def kernel_matrix(X, kernel, **kwargs):
    n = X.shape[0]
    K = np.zeros((n, n))
    for i in range(n):
        # print(i)
        for j in range(n):
            # if j % 1000 == 0:
            #     print('----------------', j)
            K[i, j] = kernel(X[i], X[j], **kwargs)
    return K


def scipy_sparse_to_spmatrix(A):
    coo = A.tocoo()
    SP = spmatrix(coo.data.tolist(), coo.row.tolist(), coo.col.tolist(), size=A.shape)
    return SP

In [180]:
import numpy as np
import cvxopt
from cvxopt import matrix, solvers
import pandas as pd
from numpy import diag
from numpy.linalg.linalg import matrix_rank, cholesky


class KernelSVM:
    def __init__(self, K=None, kernel_fun=None, C=300, **kwargs):
        self.given_kernel = False
        if K is None and kernel_fun is None:
            print("Give K or kernel_fun")

        if K is not None:
            self.given_kernel = True
        self.kernel_fun = kernel_fun
        self.parameters = kwargs
        self.C = C
        self.K = K
        self.sol = None
        self.alpha_y = None
        self.support_ = None
        self.support_vectors_ = None
        self.alphas = None

    def fit(self, X_train, y_train):
        # Need X_test to compute the kernel
        if not self.given_kernel:
            self.compute_kernel(X_train)
        Q, p, G, h = self.transform_dual(y_train)
        # print(type(Q), np.array(Q))
        # print('------------------')        
        # print(matrix(p))
        # print('------------------')        
        # print(type(G), G.size)
        # print('------------------')        
        # print(matrix(h))
        # print('------------------')

        self.sol = solvers.qp(Q, p, G, h)
        self.alphas = pd.Series(self.sol['x'], index=X_train.index)
        self.alpha_y = pd.Series(np.multiply(self.alphas, (np.array(y_train))), index=X_train.index)

        self.support_ = self.alpha_y[(10 ** (-3) < self.alpha_y) & (self.alpha_y < self.C - 10 ** (-3))].index
        self.support_vectors_ = X_train.loc[self.support_]
        # self.alphas = pd.Series(self.sol['x'], index=X_train.index)
        self.alphas = self.alphas.loc[self.support_]
        self.alpha_y = self.alphas.loc[self.support_]

        
    def predict(self, X_test):
        """Return predictions for test data"""
        return self.predict_margin(X_test).apply(np.sign)

    def predict_margin(self, X_test):
        """Return prediction margins for test data"""
        return X_test.apply(self.predict_x, axis=1)

    def predict_x(self, x):
        """Return predictions for a vector"""
        if self.given_kernel:
            return self.alphas.dot(self.K[self.support_, x])
        # return self.alphas.dot(
        #     pd.Series([self.kernel_fun(support_vec, x) for _, support_vec in self.support_vectors_.iterrows()],
        #               index=self.support_))
        a = self.alpha_y.values,
        b = np.array([np.vdot(support_vec, x) for _, support_vec in self.support_vectors_.iterrows()])
        print(a)
        print('----------------------')
        print(b)
        return np.vdot(a,b)
    

    def compute_kernel(self, X):
        n = X.shape[0]
        # self.K = pd.DataFrame(np.zeros((n, n)), index=X.index, columns=X.index)
        self.K = np.zeros((n, n))
        print(self.K.shape)
        # for i, row1 in X.iterrows():
        #     for j, row2 in X.iterrows():
        #         # print(self.kernel_fun(row1, row2, **self.parameters))
        #         # self.K.loc[i, j] = self.kernel_fun(row1, row2, **self.parameters)
        for i in range(n):
            row1 = X[i]
            for j in range(n):
                row2 = X[j]
                # self.K[i, j] = self.kernel_fun(row1, row2, **self.parameters)
                self.K[i, j] = np.vdot(row1, row2)


    def transform_dual(self, y):
        # Here, we keep lambda as the variable. We need to make A bigger to account for the 2 inequalities on lambda
        n = self.K.shape[0]
        P = self.K
        q = - y.values * np.ones(len(y))
        G = np.zeros((2 * n, n))
        h = np.zeros(2 * n)
        G[:n, :] = matrix(np.diag(y))
        G[n:2 * n, :] = matrix(-np.diag(y))
        h[:n] = self.C * np.ones(n)
        # print(type(P), P)
        # print('------------------')        
        # print(matrix(q))
        # print('------------------')        
        # print(matrix(G))
        # print('------------------')        
        # print(matrix(h))
        # print('------------------')
        return matrix(P), matrix(q), matrix(G), matrix(h)

In [126]:
train = pd.read_csv('../data/Xtr.csv', sep = ",", header = None)
trainOutput = pd.read_csv('../data/Ytr.csv', sep = ",")
test = pd.read_csv('../data/Xte.csv', sep = ",", header = None)

In [126]:
train.drop(train.columns[len(train.columns)-1], axis=1, inplace=True)
test.drop(test.columns[len(test.columns)-1], axis=1, inplace=True)

In [181]:
class multiclassSVM:
    
    def __init__(self, K=None, kernel_fun=None, C=300, **kwargs):
        self.kernel_fun = kernel_fun
        self.parameters = kwargs
        self.C = C
        self.K = K
        self.models = {}
        self.classes = None

    def fit(self, X_train, y_train):
        
        classes = np.unique(y_train)
        self.classes = classes
        for l1 in range(len(classes)):
            for l2 in range(l1+1,len(classes)):
                self.models[classes[l1],classes[l2]] = KernelSVM(K=self.K, kernel_fun=self.kernel_fun, C=self.C)
                X_train_local = X_train[(y_train == classes[l1]) + (y_train == classes[l2])]
                y_train_local = y_train[(y_train == classes[l1]) + (y_train == classes[l2])]
                y_train_local = 2*(y_train_local == classes[l1]) - 1
                self.models[classes[l1],classes[l2]].fit(X_train_local, y_train_local)
        
        
    def predict(self, X_test):
        
        ensemble_predictions = np.empty([len(X_test), 0])
        for k in self.models.keys():
            local_prediction = self.models[k].predict(X_test)
            ensemble_predictions = np.hstack((ensemble_predictions, np.array([k[0]*(local_prediction == 1) + k[1]*(local_prediction == -1)]).T))
            
        sum_array = np.array([np.sum(ensemble_predictions == c, axis = 1) for c in self.classes]).T
        final_predictions = self.classes[np.argmax(sum_array, axis=1)]
            
        return final_predictions, ensemble_predictions, sum_array

In [128]:
length = 100

ttrain = train[:length]
ttrainOutput = trainOutput[:length]['Prediction']
ttest = test[:length//2]
K = kernel_matrix(ttrain, linear_kernel)

In [182]:
mSVM = multiclassSVM(kernel_fun = linear_kernel)

In [185]:
mSVM.fit(ttrain, ttrainOutput)

(14, 14)
     pcost       dcost       gap    pres   dres
 0: -1.2962e+03 -1.2138e+04  1e+04  0e+00  8e-14
 1: -1.9269e+03 -3.3912e+03  1e+03  1e-16  7e-15
 2: -2.5194e+03 -2.9353e+03  4e+02  1e-16  2e-15
 3: -2.6441e+03 -2.6761e+03  3e+01  2e-16  7e-16
 4: -2.6579e+03 -2.6586e+03  6e-01  2e-16  6e-16
 5: -2.6582e+03 -2.6582e+03  7e-03  2e-16  7e-16
 6: -2.6582e+03 -2.6582e+03  7e-05  2e-16  7e-16
Optimal solution found.
(17, 17)
     pcost       dcost       gap    pres   dres
 0: -1.7554e+03 -1.6034e+04  1e+04  0e+00  8e-14
 1: -2.6126e+03 -4.9316e+03  2e+03  1e-16  1e-14
 2: -3.4668e+03 -3.8613e+03  4e+02  1e-16  2e-15
 3: -3.6573e+03 -3.7178e+03  6e+01  2e-16  9e-16
 4: -3.6839e+03 -3.6887e+03  5e+00  2e-16  1e-15
 5: -3.6862e+03 -3.6864e+03  2e-01  2e-16  9e-16
 6: -3.6863e+03 -3.6863e+03  2e-03  2e-16  2e-15
Optimal solution found.
(18, 18)


     pcost       dcost       gap    pres   dres
 0: -5.4365e+02 -2.0307e+04  2e+04  0e+00  1e-13
 1: -1.8057e+03 -4.7076e+03  3e+03  2e-16  1e-14
 2: -2.5816e+03 -3.1088e+03  5e+02  1e-16  2e-15
 3: -2.7614e+03 -2.8611e+03  1e+02  2e-16  8e-16
 4: -2.7983e+03 -2.8038e+03  5e+00  1e-16  8e-16
 5: -2.8003e+03 -2.8005e+03  3e-01  2e-16  2e-15
 6: -2.8004e+03 -2.8004e+03  2e-02  1e-16  9e-16
 7: -2.8004e+03 -2.8004e+03  1e-03  2e-16  8e-16
Optimal solution found.
(15, 15)
     pcost       dcost       gap    pres   dres
 0:  1.5334e+02 -2.1245e+04  2e+04  0e+00  7e-14
 1: -1.1601e+03 -3.5489e+03  2e+03  1e-16  6e-15
 2: -1.8351e+03 -2.6410e+03  8e+02  1e-16  1e-15
 3: -2.0442e+03 -2.1916e+03  1e+02  2e-16  5e-16
 4: -2.1042e+03 -2.1296e+03  3e+01  2e-16  6e-16
 5: -2.1150e+03 -2.1160e+03  1e+00  2e-16  1e-15
 6: -2.1155e+03 -2.1155e+03  2e-02  2e-16  1e-15
 7: -2.1155e+03 -2.1155e+03  2e-04  2e-16  1e-15
Optimal solution found.


/home/nico/anaconda3/lib/python3.5/site-packages/pandas/computation/expressions.py:182: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


(19, 19)
     pcost       dcost       gap    pres   dres
 0:  2.3298e+03 -5.6353e+04  6e+04  0e+00  1e-13
 1: -1.8314e+03 -8.2788e+03  6e+03  1e-16  9e-15
 2: -2.5070e+03 -4.3567e+03  2e+03  1e-16  3e-15
 3: -2.8577e+03 -3.5307e+03  7e+02  2e-16  2e-15
 4: -3.0219e+03 -3.1253e+03  1e+02  1e-16  3e-15
 5: -3.0628e+03 -3.0731e+03  1e+01  2e-16  2e-15
 6: -3.0665e+03 -3.0679e+03  1e+00  2e-16  2e-15
 7: -3.0670e+03 -3.0671e+03  4e-02  2e-16  3e-15
 8: -3.0670e+03 -3.0670e+03  4e-04  2e-16  3e-15
Optimal solution found.
(16, 16)
     pcost       dcost       gap    pres   dres
 0: -1.1518e+03 -1.3152e+04  1e+04  0e+00  7e-14
 1: -2.0500e+03 -4.2604e+03  2e+03  2e-16  1e-14
 2: -2.7334e+03 -3.3090e+03  6e+02  2e-16  2e-15
 3: -2.8788e+03 -2.9707e+03  9e+01  2e-16  8e-16
 4: -2.9164e+03 -2.9255e+03  9e+00  2e-16  7e-16
 5: -2.9191e+03 -2.9197e+03  6e-01  2e-16  7e-16
 6: -2.9193e+03 -2.9193e+03  6e-03  2e-16  7e-16
 7: -2.9193e+03 -2.9193e+03  6e-05  2e-16  5e-16
Optimal solution found.
(14, 

(16, 16)
     pcost       dcost       gap    pres   dres
 0: -2.5799e+02 -1.9712e+04  2e+04  0e+00  8e-14
 1: -1.3641e+03 -3.4329e+03  2e+03  1e-16  5e-15
 2: -1.6307e+03 -2.0360e+03  4e+02  2e-16  7e-16
 3: -1.7667e+03 -1.8475e+03  8e+01  2e-16  8e-16
 4: -1.7940e+03 -1.8026e+03  9e+00  1e-16  8e-16
 5: -1.7960e+03 -1.7967e+03  7e-01  2e-16  6e-16
 6: -1.7962e+03 -1.7962e+03  1e-02  2e-16  8e-16
 7: -1.7962e+03 -1.7962e+03  1e-04  2e-16  8e-16
Optimal solution found.
(14, 14)


...

(19, 19)
     pcost       dcost       gap    pres   dres
 0: -4.9400e+02 -1.8653e+04  2e+04  0e+00  9e-14
 1: -1.6206e+03 -4.2986e+03  3e+03  1e-16  9e-15
 2: -2.1617e+03 -2.7773e+03  6e+02  2e-16  3e-15
 3: -2.3828e+03 -2.4690e+03  9e+01  2e-16  2e-15
 4: -2.4102e+03 -2.4146e+03  4e+00  2e-16  2e-15
 5: -2.4122e+03 -2.4124e+03  2e-01  2e-16  2e-15
 6: -2.4123e+03 -2.4123e+03  1e-02  2e-16  3e-15
 7: -2.4123e+03 -2.4123e+03  2e-04  2e-16  2e-15
Optimal solution found.
(23, 23)
     pcost       dcost       gap    pres   dres
 0: -1.0435e+01 -5.2799e+04  5e+04  0e+00  1e-13
 1: -2.2106e+03 -9.0266e+03  7e+03  2e-16  1e-14
 2: -3.5003e+03 -4.9201e+03  1e+03  2e-16  4e-15
 3: -4.0029e+03 -4.2398e+03  2e+02  2e-16  3e-15
 4: -4.0718e+03 -4.0977e+03  3e+01  2e-16  3e-15
 5: -4.0825e+03 -4.0831e+03  7e-01  2e-16  4e-15
 6: -4.0827e+03 -4.0827e+03  7e-03  2e-16  4e-15
 7: -4.0827e+03 -4.0827e+03  7e-05  2e-16  6e-15
Optimal solution found.
(20, 20)
     pcost       dcost       gap    pres   dr

     pcost       dcost       gap    pres   dres
 0:  3.7896e+03 -8.4540e+04  9e+04  0e+00  7e-14
 1: -4.7674e+02 -1.0778e+04  1e+04  1e-16  7e-15
 2: -1.6813e+03 -3.9336e+03  2e+03  1e-16  3e-15
 3: -2.1025e+03 -3.1127e+03  1e+03  1e-16  2e-15
 4: -2.3581e+03 -2.5636e+03  2e+02  2e-16  3e-15
 5: -2.3970e+03 -2.4705e+03  7e+01  2e-16  4e-15
 6: -2.4223e+03 -2.4262e+03  4e+00  2e-16  4e-15
 7: -2.4238e+03 -2.4239e+03  4e-02  2e-16  4e-15
 8: -2.4238e+03 -2.4238e+03  4e-04  1e-16  4e-15
Optimal solution found.
(17, 17)
     pcost       dcost       gap    pres   dres
 0: -7.6278e+01 -1.6846e+04  2e+04  0e+00  7e-14
 1: -1.2286e+03 -3.5760e+03  2e+03  2e-16  7e-15


 2: -1.7710e+03 -2.2196e+03  4e+02  1e-16  1e-15
 3: -1.9485e+03 -2.0199e+03  7e+01  2e-16  9e-16
 4: -1.9765e+03 -1.9816e+03  5e+00  1e-16  7e-16
 5: -1.9783e+03 -1.9785e+03  2e-01  2e-16  8e-16
 6: -1.9783e+03 -1.9783e+03  3e-03  2e-16  1e-15
 7: -1.9783e+03 -1.9783e+03  3e-05  2e-16  9e-16
Optimal solution found.
(15, 15)
     pcost       dcost       gap    pres   dres
 0:  1.9236e+02 -1.9715e+04  2e+04  0e+00  9e-14
 1: -1.1073e+03 -4.2619e+03  3e+03  1e-16  1e-14
 2: -1.9098e+03 -2.5089e+03  6e+02  2e-16  2e-15
 3: -2.0918e+03 -2.1950e+03  1e+02  2e-16  7e-16
 4: -2.1282e+03 -2.1357e+03  8e+00  1e-16  7e-16
 5: -2.1313e+03 -2.1315e+03  2e-01  2e-16  9e-16
 6: -2.1314e+03 -2.1314e+03  3e-03  2e-16  9e-16
 7: -2.1314e+03 -2.1314e+03  3e-05  3e-16  6e-16
Optimal solution found.
(25, 25)
     pcost       dcost       gap    pres   dres
 0:  7.1973e+03 -9.8943e+04  1e+05  0e+00  8e-14
 1: -1.2996e+03 -1.4185e+04  1e+04  1e-16  9e-15
 2: -2.7328e+03 -5.3710e+03  3e+03  1e-16  2e-15
 3: -

     pcost       dcost       gap    pres   dres
 0: -9.4708e+02 -5.2143e+04  5e+04  0e+00  7e-14
 1: -3.0462e+03 -8.5388e+03  5e+03  2e-16  7e-15
 2: -4.3743e+03 -5.4865e+03  1e+03  1e-16  4e-15
 3: -4.7353e+03 -4.9009e+03  2e+02  2e-16  3e-15
 4: -4.7981e+03 -4.8091e+03  1e+01  2e-16  4e-15
 5: -4.8035e+03 -4.8037e+03  3e-01  2e-16  4e-15
 6: -4.8036e+03 -4.8036e+03  3e-03  2e-16  4e-15
Optimal solution found.
(16, 16)


     pcost       dcost       gap    pres   dres
 0: -7.5701e+02 -2.1745e+04  2e+04  0e+00  9e-14
 1: -1.7901e+03 -4.4874e+03  3e+03  1e-16  9e-15
 2: -2.7014e+03 -3.2618e+03  6e+02  2e-16  1e-15
 3: -2.9096e+03 -2.9578e+03  5e+01  1e-16  7e-16
 4: -2.9289e+03 -2.9301e+03  1e+00  2e-16  7e-16
 5: -2.9294e+03 -2.9294e+03  5e-02  3e-16  9e-16
 6: -2.9294e+03 -2.9294e+03  4e-03  2e-16  7e-16
 7: -2.9294e+03 -2.9294e+03  6e-05  2e-16  1e-15
Optimal solution found.
(26, 26)
     pcost       dcost       gap    pres   dres
 0:  7.9291e+03 -8.2130e+04  9e+04  0e+00  9e-14
 1: -9.7316e+01 -1.0540e+04  1e+04  1e-16  6e-15
 2: -1.7879e+03 -3.9824e+03  2e+03  1e-16  1e-15
 3: -2.1994e+03 -2.7830e+03  6e+02  2e-16  6e-16
 4: -2.3363e+03 -2.5147e+03  2e+02  1e-16  8e-16
 5: -2.3770e+03 -2.4059e+03  3e+01  2e-16  8e-16
 6: -2.3846e+03 -2.3891e+03  5e+00  2e-16  8e-16
 7: -2.3860e+03 -2.3863e+03  3e-01  2e-16  7e-16
 8: -2.3861e+03 -2.3861e+03  2e-02  2e-16  7e-16
 9: -2.3861e+03 -2.3861e+03  8e-04  2

In [186]:
for k, model in mSVM.models.items():
    print(k, len(model.support_vectors_))

(5, 9) 13
(4, 7) 6
(1, 3) 8
(4, 8) 10
(5, 6) 8
(2, 8) 12
(6, 9) 8
(0, 7) 5
(4, 6) 8
(8, 9) 10
(1, 6) 9
(3, 7) 9
(2, 5) 11
(0, 3) 8
(5, 8) 12
(1, 2) 7
(6, 7) 6
(2, 9) 11
(1, 5) 7
(3, 6) 11
(0, 4) 4
(7, 9) 9
(0, 1) 5
(2, 6) 5
(4, 5) 8
(1, 4) 5
(3, 9) 10
(2, 3) 9
(1, 9) 9
(4, 9) 8
(0, 8) 10
(3, 5) 11
(2, 7) 6
(6, 8) 14
(3, 4) 9
(5, 7) 6
(2, 4) 9
(3, 8) 8
(0, 6) 6
(1, 8) 12
(1, 7) 6
(0, 9) 8
(0, 5) 9
(7, 8) 13
(0, 2) 5


In [187]:
final_predictions, ensemble_predictions, sum_array = mSVM.predict(ttest)

(array([-177.23371447,  101.01166236,   70.23193302, -299.99073334,
       -238.36915595, -211.36866918, -138.93024745, -299.99835504,
        212.71726105,  147.1107696 , -198.00150339,  108.81714565,
        211.04859611]),)
----------------------
[ 0.09419207  0.05678304 -0.23284227 -0.08697587  0.50851218  0.11216939
  0.62755544  0.41046237 -0.00612384  0.16766768  0.19801842 -0.11686376
  0.16596721]
(array([-177.23371447,  101.01166236,   70.23193302, -299.99073334,
       -238.36915595, -211.36866918, -138.93024745, -299.99835504,
        212.71726105,  147.1107696 , -198.00150339,  108.81714565,
        211.04859611]),)
----------------------
[-0.17244492 -0.27289364 -0.02138107 -0.29175605  0.08958759 -0.09216344
 -0.03876557  0.05674137  0.06081239 -0.13082652  0.10198686  0.02969009
 -0.02921259]
(array([-177.23371447,  101.01166236,   70.23193302, -299.99073334,
       -238.36915595, -211.36866918, -138.93024745, -299.99835504,
        212.71726105,  147.1107696 , -198.001

(array([ -6.71588298e+01,  -2.15780263e+02,  -4.72661837e-03,
        -1.05652789e+02,  -1.65962471e+02,   9.21576303e+01,
        -2.03253221e+02,   1.39481326e+02]),)
----------------------
[-0.04191619 -0.07399346  0.00398789 -0.21971298  0.08512111  0.14691451
  0.06983947  0.03112674]
(array([ -6.71588298e+01,  -2.15780263e+02,  -4.72661837e-03,
        -1.05652789e+02,  -1.65962471e+02,   9.21576303e+01,
        -2.03253221e+02,   1.39481326e+02]),)
----------------------
[ 0.14338486  0.20363984  0.47438793 -0.08275979 -0.0506284   0.01453136
  0.05669066  0.03966739]
(array([ -6.71588298e+01,  -2.15780263e+02,  -4.72661837e-03,
        -1.05652789e+02,  -1.65962471e+02,   9.21576303e+01,
        -2.03253221e+02,   1.39481326e+02]),)
----------------------
[-0.02383677 -0.27974186 -0.02683048  0.04251455  0.02825127 -0.03637966
 -0.16913751  0.2170605 ]
(array([ -6.71588298e+01,  -2.15780263e+02,  -4.72661837e-03,
        -1.05652789e+02,  -1.65962471e+02,   9.21576303e+01,
    


[-0.02312624 -0.15385348 -0.12311995  0.20674658  0.05203493  0.1274839
 -0.12011875  0.13091819 -0.15756114  0.22788278  0.18164176 -0.10820598]
(array([ -28.80476613, -220.95075719,  128.47126399, -115.47685897,
        -97.83367644, -194.15275129, -235.68613029, -170.96124012,
        -50.41859143,  146.98887791,  123.0250676 ,  296.5412701 ]),)
----------------------
[ 0.19995432  0.05848671 -0.01519511 -0.11487432 -0.0932829   0.1064674
 -0.09968966 -0.01042367 -0.00816049 -0.02762098 -0.23911006 -0.00176469]
(array([ -28.80476613, -220.95075719,  128.47126399, -115.47685897,
        -97.83367644, -194.15275129, -235.68613029, -170.96124012,
        -50.41859143,  146.98887791,  123.0250676 ,  296.5412701 ]),)
----------------------
[ 0.07597117 -0.08604209  0.00944206  0.2516265   0.00226     0.37384452
 -0.09554182 -0.0559019   0.03153174 -0.12611258  0.10057789  0.15741577]
(array([ -28.80476613, -220.95075719,  128.47126399, -115.47685897,
        -97.83367644, -194.15275129,

(array([  74.89159054, -236.23091176, -155.04254442, -245.82246678,
       -215.87736849]),)
----------------------
[-0.26888917  0.06426152 -0.06788961 -0.03143268  0.01042749]
(array([  74.89159054, -236.23091176, -155.04254442, -245.82246678,
       -215.87736849]),)
----------------------
[ 0.14436738 -0.04421098 -0.11024049 -0.13977866 -0.01752814]
(array([  74.89159054, -236.23091176, -155.04254442, -245.82246678,
       -215.87736849]),)
----------------------
[-0.25780018 -0.11376076 -0.26497814 -0.05112457 -0.15869313]
(array([  74.89159054, -236.23091176, -155.04254442, -245.82246678,
       -215.87736849]),)
----------------------
[ 0.08577243  0.10865583  0.25244814  0.2700535  -0.18154218]
(array([  74.89159054, -236.23091176, -155.04254442, -245.82246678,
       -215.87736849]),)
----------------------
[ 0.01654311 -0.10824057 -0.01003452 -0.02017587 -0.07553676]
(array([  74.89159054, -236.23091176, -155.04254442, -245.82246678,
       -215.87736849]),)
-----------------


----------------------
[-0.02882858  0.06634489 -0.04656991  0.10855145 -0.12325408  0.1283785
 -0.01059878 -0.05992723 -0.23320271]
(array([  47.25635316,  -33.32093971, -125.95937507, -177.89294627,
        -39.46717216,  166.07137313,  -15.24734685, -266.77424423,
        100.93187753]),)
----------------------
[ 0.21115718 -0.15285172  0.14612404 -0.25593987 -0.06342317  0.04759258
  0.13884274 -0.13068098 -0.14008649]
(array([  47.25635316,  -33.32093971, -125.95937507, -177.89294627,
        -39.46717216,  166.07137313,  -15.24734685, -266.77424423,
        100.93187753]),)
----------------------
[ 0.19191896  0.13881773  0.04572556  0.3645051  -0.03749237 -0.0886538
 -0.07519847  0.01153182  0.04021426]
(array([  47.25635316,  -33.32093971, -125.95937507, -177.89294627,
        -39.46717216,  166.07137313,  -15.24734685, -266.77424423,
        100.93187753]),)
----------------------
[ 0.23447151 -0.15419622  0.15341588  0.02391837 -0.04962197  0.10966439
 -0.10665572  0.0312425

...

(array([ -2.64156401e+01,  -2.37563632e+02,  -3.73207259e-02,
        -9.75434477e+01,   2.97254105e+02,   2.21334894e+02,
        -7.79965856e+01,  -2.30015431e+01,  -5.75631601e-02,
        -2.57423888e+01,  -6.40375856e+01,  -1.13106424e+02,
         7.33308470e-03,   2.26924181e+02]),)
----------------------
[ 0.00341462  0.37298177  0.14254019  0.04249188 -0.3108583   0.01935336
  0.04644157 -0.23876349 -0.15984115 -0.19025404  0.02349659  0.01713738
 -0.24687806 -0.0088419 ]
(array([ -2.64156401e+01,  -2.37563632e+02,  -3.73207259e-02,
        -9.75434477e+01,   2.97254105e+02,   2.21334894e+02,
        -7.79965856e+01,  -2.30015431e+01,  -5.75631601e-02,
        -2.57423888e+01,  -6.40375856e+01,  -1.13106424e+02,
         7.33308470e-03,   2.26924181e+02]),)
----------------------
[ 0.10242484  0.11782095  0.20618669  0.0270468  -0.06973566  0.03835426
  0.18362661 -0.03233541 -0.01004828  0.07196065 -0.11286562 -0.08826956
 -0.11000842  0.01239578]
(array([ -2.64156401e+01,  -

(array([ -82.87850748,  272.27884822,  102.8887739 ,  224.58572136,
        -60.24353649,  229.27179473,   31.79193882, -213.66159197,
         45.62973811]),)
----------------------
[-0.45780288 -0.12597206 -0.0788095   0.18746396  0.02855316 -0.06049551
 -0.03338117  0.03360046  0.05818984]
(array([ -82.87850748,  272.27884822,  102.8887739 ,  224.58572136,
        -60.24353649,  229.27179473,   31.79193882, -213.66159197,
         45.62973811]),)
----------------------
[-0.09659835 -0.22931795  0.09181398 -0.1798641   0.02238783 -0.24567289
  0.12907746 -0.09175968 -0.08200616]
(array([ -82.87850748,  272.27884822,  102.8887739 ,  224.58572136,
        -60.24353649,  229.27179473,   31.79193882, -213.66159197,
         45.62973811]),)
----------------------
[ 0.15893615  0.12293534  0.08855004 -0.27652153  0.23303611  0.17251315
 -0.09600991 -0.18657451  0.00912238]
(array([ -82.87850748,  272.27884822,  102.8887739 ,  224.58572136,
        -60.24353649,  229.27179473,   31.79193882

[ 0.04652059 -0.18471221  0.00152147  0.05928037  0.23005925 -0.02652578
  0.23365037  0.0069907  -0.15531178 -0.07557542  0.29683359 -0.12837289]
(array([-138.67927607,  247.62619884,  -91.74038525,  -15.8869007 ,
       -234.46988632, -159.57628397, -177.0476445 ,  -53.06801401,
       -290.71923481, -171.59046671, -127.9484656 , -299.99864638]),)
----------------------
[ 0.04917025 -0.16023105 -0.02312624  0.06603483  0.20674658  0.1274839
 -0.12941073 -0.12011875 -0.27164672  0.00835784 -0.05818824  0.13916442]
(array([-138.67927607,  247.62619884,  -91.74038525,  -15.8869007 ,
       -234.46988632, -159.57628397, -177.0476445 ,  -53.06801401,
       -290.71923481, -171.59046671, -127.9484656 , -299.99864638]),)
----------------------
[ 0.13485583  0.2518242   0.19995432 -0.00069034 -0.11487432  0.1064674
  0.05413187 -0.09968966  0.04034414  0.12195127  0.21629887 -0.05122436]
(array([-138.67927607,  247.62619884,  -91.74038525,  -15.8869007 ,
       -234.46988632, -159.57628397, 


----------------------
[-0.10352591 -0.30189447 -0.17095506  0.00215928  0.36899479  0.16610568
 -0.21376181 -0.04143421]
(array([-269.49671464, -164.31868256, -110.09025777,  -85.16285136,
       -192.31083234,   99.36470293, -226.00303192,  118.27081478]),)
----------------------
[-0.05507806  0.24320346 -0.05367332  0.34068372 -0.26432947 -0.11821248
  0.02767183 -0.00659049]
(array([-269.49671464, -164.31868256, -110.09025777,  -85.16285136,
       -192.31083234,   99.36470293, -226.00303192,  118.27081478]),)
----------------------
[ 0.01340816  0.14796163  0.00768535 -0.57417805 -0.63643834 -0.01945323
 -0.09269087 -0.06659334]
(array([-269.49671464, -164.31868256, -110.09025777,  -85.16285136,
       -192.31083234,   99.36470293, -226.00303192,  118.27081478]),)
----------------------
[-0.09284533 -0.39348823  0.1944809  -0.35285761 -0.20119449  0.35123892
  0.03555988  0.16679687]
(array([-269.49671464, -164.31868256, -110.09025777,  -85.16285136,
       -192.31083234,   99.36


(array([-198.41913575,  299.98253178, -209.74083349, -247.02303639,
       -288.88748539]),)
----------------------
[ 0.13713917  0.18441605  0.19292172 -0.03476207 -0.11286107]
(array([-198.41913575,  299.98253178, -209.74083349, -247.02303639,
       -288.88748539]),)
----------------------
[-0.01167928  0.13667182 -0.06529835 -0.00374784  0.2356123 ]
(array([-198.41913575,  299.98253178, -209.74083349, -247.02303639,
       -288.88748539]),)
----------------------
[ 0.09596174 -0.03081258  0.23394736 -0.14312819 -0.17732688]
(array([-198.41913575,  299.98253178, -209.74083349, -247.02303639,
       -288.88748539]),)
----------------------
[-0.12311995  0.17601586  0.25212719  0.13750055  0.12593602]
(array([-198.41913575,  299.98253178, -209.74083349, -247.02303639,
       -288.88748539]),)
----------------------
[-0.01519511 -0.04511033 -0.04966266  0.03822146 -0.14100436]
(array([-198.41913575,  299.98253178, -209.74083349, -247.02303639,
       -288.88748539]),)
----------------

In [188]:
ens_pred[0]

NameError: name 'ens_pred' is not defined

In [189]:
sum_array[0]

array([6, 6, 5, 4, 3, 3, 1, 2, 7, 8])

In [190]:
len(final_predictions)

50